# Extending PointNet Networks and Simulation with External and Customized Models.

In this tutorial, we explain through a series of examples how to use and simulate built-in NEST cell and synaptic models, and create and import custom models using NESTML, as well as ways to dynamically change cells parametters before each simulation.

**Note** - scripts and files for running this tutorial can be found in the directory [02_opt_nest_custom_models/](https://github.com/AllenInstitute/bmtk/tree/develop/docs/tutorial/02_opt_nest_custom_models)

---

## Contents
1. [Example: Using built-in NEST models](#example-advance-icalmp)
2. [Example: Custom cell models with NESTML](#example-nestml)
3. [Example: Overriding the creation of cell models](#example-cell-creation)
4. [Example: Switching between synaptic models](#example-switching-models)

---

## 1. Example: Using Built-In NEST Models <a class="anchor" id="example-nest-models">

PointNet utlizes the NEST simulator for running point-neuron simulations, which includes many different varieties and types of neuronal models the majority of which can readily used during simulation in BMTK. An very useful practice is to take an existing network model built on model of a cell and replace it with a different cell model that more closely fits the needs of our research. With BMTK and SONATA this can be easily done, often without any programming, and even done on the fly before any given simulation.

A list of available models built into the NEST simulator can be found [here](https://nest-simulator.readthedocs.io/en/stable/models/index.html).

Most of the spiking models will work out-of-the-box in BMTK, readily swapped in and out of existing simulations, and you can use multiple models in the same network. (*We will ignore rates-based models which can only tentively work with BMTK. And for devices like genators and recorders PointNet uses a separate way of inserting them into the simulation*). To define what model will be used with a specific subset of cells and their properties, SONATA has two defined reserved attributes `model_template` and `dynamics_params` which we will take advantage of in this example.

### Setting the model type in our network.

The default way to set the model type for subset of neurons (which may be just a single neuron) that all use the same built-in NEST model type is to set the the **model_type** attributes to a value `nest:<nest-model-name>` where `<nest-model-name>` is the name of one of the spiking [NEST cell models](https://nest-simulator.readthedocs.io/en/stable/models/index.html). However, different models will have different parameters. There are multiple ways to set model parameters in BMTK, but the one that is most flexible and will make it easier to switch out different models while testing and optimizing our network is to set the **dynamics_params** to an easy to edit json file.

In the below example (*build_network.built_in.py*) will built a simple 100 cell network with synaptic stimuli. Initialally we will use [izhikevich](https://nest-simulator.readthedocs.io/en/stable/models/izhikevich.html) models using default params (**hint**: if the dynamics_params .json file is just an empty json file, then the cells will be loaded with the default params. This is a useful starting point). But you can change the **nest_model** (and optionally the **dynamics_params**) variable below to build the same network using different model types

In [1]:
nest_model = 'nest:izhikevich'
# nest_model = 'nest:iaf_psc_delta'
# nest_model = 'nest:aeif_cond_alpha'
# nest_model = 'nest:glif_psc'

dynamics_params = 'custom_model_params.default.json'
# dynamics_params = 'custom_model_params.izhikevich.json'
# dynamics_params = 'custom_model_params.aeif_cond_alpha.json'
# dynamics_params = 'custom_model_params.glif_psc.json'

In [2]:
import numpy as np
from bmtk.builder import NetworkBuilder


net = NetworkBuilder('net')
net.add_nodes(
    N=100,
    model_type='point_neuron',
    model_template=nest_model,
    dynamics_params=dynamics_params
)

net.add_edges(
    source=net.nodes(), target=net.nodes(),
    connection_rule=1,
    syn_weight=2.0,
    delay=1.5,
    dynamics_params='ExcToInh.json',
    model_template='static_synapse'
)

net.build()
net.save(output_dir='network_built_in')


virt_exc = NetworkBuilder('virt_exc')
virt_exc.add_nodes(
    N=10,
    model_type='virtual'
)

virt_exc.add_edges(
    target=net.nodes(),
    connection_rule=lambda *_: np.random.randint(0, 10),
    syn_weight=2.0,
    delay=1.0,
    dynamics_params='ExcToInh.json',
    model_template='static_synapse'
)

virt_exc.build()
virt_exc.save(output_dir='network_built_in')

[OSXLTC5747Y:45465] shmem: mmap: an error occurred while determining whether or not /var/folders/pr/_10c7kr546b1t7cf9z259z_40000gp/T//ompi.OSXLTC5747Y.502/jf.0/3216834560/sm_segment.OSXLTC5747Y.502.bfbd0000.0 could be created.


If you have access to an existing model but not the build-script, or if rebuilding the network is too time consuming and/or expensive, then another option to swap out different models is to open the *node_types.csv* in an text editor (especially ones that support csv forats like VSCode or Atom), update the **model_template** and **dynamics_params** attributes, then rerun the simulation.

In [3]:
import pandas as pd

pd.read_csv('network_built_in/net_node_types.csv', sep=' ')

,node_type_id,dynamics_params,model_type,model_template
0,100,custom_model_params.default.json,point_neuron,nest:izhikevich


We encourage you to try running the network yourself by either running in the command line or un
```
$ python run_pointnet.built_in.py config.built_in.json
```
or with the below cell, trying it with different cell models. 

In [4]:
from bmtk.simulator import pointnet

configure = pointnet.Config.from_json('config.built_in.json')
configure.build_env()

network = pointnet.PointNetwork.from_config(configure)
sim = pointnet.PointSimulator.from_config(configure, network)
sim.run()


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: May 24 2024 10:13:36

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

2024-10-21 15:11:35,805 [INFO] Created log file
2024-10-21 15:11:35,813 [INFO] Batch processing nodes for net/0.
2024-10-21 15:11:35,816 [INFO] Batch processing nodes for virt_exc/0.
2024-10-21 15:11:35,827 [INFO] Setting up output directory
2024-10-21 15:11:35,827 [INFO] Building cells.
2024-10-21 15:11:35,830 [INFO] Building recurrent connections
2024-10-21 15:11:35,834 [INFO] Network created.
2024-10-21 15:11:35,834 [INFO] Build virtual cell stimulations for thalamus_spikes
2024-10-21 15:11:35,844 [INFO] Starting Simulation
2024-10-21 15:11:39,062 [INFO] Simulation finished, finalizing results.
2024-10-21 15:11:39,351 [INFO] Done.


One thing to note is that when you replace one cell-model with another you may often get widely varying results. A network that is stable or silent in using one cell type may explode when a model gets replaced. Fixing this may require the or both of the following following:
* Adjusting the **dynamics_params** of the new cell type.
* Adjust the synaptic models and parameters (especially **syn_weight**).

## 2. Example: Importing Custom NESTML cell models <a class="anchor" id="example-nestml">

If you aren't able to find a signficant cell model that will work for your network already built into NEST, then one option is to create it yourself (or import from an existing model) using [NESTML](https://nestml.readthedocs.io/en/latest/). NESTML is a library to build, compile, and insert customized NEST cell and synapse models into a simulation. NESTML provides a very simple language for building all kinds of cell models. And these models can be readily inserted into BMTK with only a few minor changes to the SONATA configuration files.

#### Note

To build cell models using NESTML it requires [installing the python package](https://nestml.readthedocs.io/en/latest/installation.html). While NESTML does have a PyPI (eg `pip`) option for installing the package, due to its rapid changes we recommend that for the time being install directly from github. Otherwise some. As of at-least version 7.0.2 (April 2024) the instructions below and in the official tutorials has issues with the PyPI install.

### Building the models

First step is to create and compile the NESTML cell model(s). This usually requires creating a `.nestml` file in a regular text editor and adding information about parameters, equations, inputs and outputs in a tab indented file. The documentation contains very [well written language documentation](https://nestml.readthedocs.io/en/latest/nestml_language/nestml_language_concepts.html) and their [github repo contains many example cell and synaptic models](https://github.com/nest/nestml/tree/master/models) which you can expand upon for your models. For simplicity we'll go ahead and reimplement the `Izhikevich` model from the tutorial. The model is implemented in the following:

In [5]:
nestml_izh_model = """
model custom_izh_neuron:

    state:
        v mV = -65 mV    # Membrane potential in mV
        u real = 0    # Membrane potential recovery variable

    equations:
        v' = (.04 * v * v / mV + 5 * v + (140 - u) * mV + (I_e * GOhm)) / ms
        u' = a * (b * v - u * mV) / (mV * ms)

    parameters:
        a real = .02    # describes time scale of recovery variable
        b real = .2    # sensitivity of recovery variable
        c mV = -65 mV    # after-spike reset value of v
        d real = 8.    # after-spike reset value of u

    input:
        spikes <- spike
        I_e pA <- continuous

    output:
        spike

    update:
        integrate_odes()

    onReceive(spikes):
        # add synaptic current
        v += spikes * mV * s

    onCondition(v >= 30mV):
        # threshold crossing
        v = c
        u += d
        emit_spike()
"""

Then we use the following to build the model

In [6]:
from pynestml.codegeneration.nest_code_generator_utils import NESTCodeGeneratorUtils


module_name, neuron_model_name = NESTCodeGeneratorUtils.generate_code_for(
    nestml_izh_model,
    module_name='nestml_izh_module',
    target_path='components/nestml'
)

print(module_name, neuron_model_name)


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: May 24 2024 10:13:36

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

[13,custom_izh_neuron_nestml, WARNING, [13:8;13:17]]: Variable 'a' has the same name as a physical unit!
[14,custom_izh_neuron_nestml, WARNING, [16:8;16:17]]: Variable 'd' has the same name as a physical unit!
[23,custom_izh_neuron_nestml, WARNING, [13:8;13:17]]: Variable 'a' has the same name as a physical unit!
[24,custom_izh_neuron_nestml, WARNING, [16:8;16:17]]: Variable 'd' has the same name as a physical unit!
CMake Warning (dev) at CMakeLists.txt:93 (project):
  cmake_minimum_required() should be called prior to this top-level project()
  call.  Please see the cmake-commands(7) manual for usage documentation of
  both commands.
This warning is for project

If successful, you should see it call `cmake` and `make` commands to compile the model in nest and create a library *.so (along with source code) in the **components/nestml/** directory.

**Note** that although in this case we passed in the model as a string into the `generate_code_for` function, you also have an option of saving the model to a *.nestml textfile and passing it in. In said case it would just be the following

```python
module_name, neuron_model_name = NESTCodeGeneratorUtils.generate_code_for(
    'nestml_izh_model.nestml',
    module_name='nestml_izh_module',
    target_path='components/nestml'
)
```


### 3. Using the NESTML model in PointNet

Once we have compiled our module (eg .so or .a file) in the `target_path` folder we can go ahead and use the models like we would with a built-in nest models. In this case we have a network pre-built (*build_network.nestml.py*) in the network_nestml/ folder. First thing we must do is make sure that our **model_template** attributes point to the correct name of the models and have corresponding parameter files.

Note that by default NESTML append the suffix "_nestml" to the name of our model. 

In [7]:
import pandas as pd

pd.read_csv('network_nestml/izh_network_node_types.csv', sep=' ')

,node_type_id,ei,model_type,dynamics_params,pop_name,model_template
0,100,e,point_neuron,custom_izh_neuron_exc.json,LIF_exc,nestml:custom_izh_neuron_nestml
1,101,i,point_neuron,custom_izh_neuron_inh.json,LIF_inh,nestml:custom_izh_neuron_nestml


We must also let PointNet know where the corresponding *.so file is containing the compiled model. In our SONATA configuration file we add the an "nest_modules" entry to the **"components"**

```json
  "components": {
      "point_neuron_models_dir": "$MODELS_DIR/cell_models",
	  "synaptic_models_dir": "$MODELS_DIR/synaptic_models",
      "nest_modules": "$MODELS_DIR/nestml/nestml_izh_module.so"
  },
```

If you have multiple NEST and/or NESTML modules you can either specify "nest_modules" as a directory

```json
    "nest_modules": "/path/to/modules/dir/"
```

And BMTK will try to find and load all library files in the folder. Or alternatively you can pass in a list of library binaries:

```json
    "nest_modules": ["/first/path/to/module1.so", "/second/path/to/module2.so", ...]
```

And run the simulation as normal

In [8]:
from bmtk.simulator import pointnet

configure = pointnet.Config.from_json('config.nestml.json')
configure.build_env()

network = pointnet.PointNetwork.from_config(configure)
sim = pointnet.PointSimulator.from_config(configure, network)
sim.run()

2024-10-21 15:11:51,097 [INFO] Created log file


INFO:Created log file


2024-10-21 15:11:51,129 [INFO] Batch processing nodes for izh_network/0.


INFO:Batch processing nodes for izh_network/0.


2024-10-21 15:11:51,143 [INFO] Batch processing nodes for virt_exc/0.


...

INFO:Starting Simulation


2024-10-21 15:11:51,875 [INFO] Simulation finished, finalizing results.


INFO:Simulation finished, finalizing results.


2024-10-21 15:11:51,893 [INFO] Done.


INFO:Done.


## 3. Example: Overriding the creation of cell models <a class="anchor" id="example-cell-creation">

Before BMTK starts a simulation, it will go ahead and call on NEST to create neurons for all cells using attributes like **model_type**, **model_template**, and **dynamics_params**. BMTK will initialize the creation of these cells for you and for the vast majority of simulations the user does not have to do anythin special other than specify the models and parameters in the SONATA file. 

However, on rare occassions you may find yourself requiring more grainular control of how PointNet initializes and creates the neurons before the simulation begins. For example, you may want to find or alter cell parameters at the start of a simulation. Some models may require a non-standard way of initializing them. Or you may need to attach Parrot or Relay neurons to a subset of your cells. 

When BMTK initializes cells for simualation it will normally call the function `loadNESTModel` function in [`bmtk.simulation.pointnet.default_setters.cell_models`](). This is a simple function that calls that NEST `Create` function to initialize a batch of neurons that share the same model-name and properites.

```python
def loadNESTModel(cell, template_name, dynamics_params):
    return nest.Create(template_name, cell.n_nodes, dynamics_params)
```

If you need to override this function, BMTK provides a special [python decorator](https://peps.python.org/pep-0318/) called `@cell_model`. To use the decorator you can add the following to your *run_pointnet.py* (or in a separate python file imported before you run the simulation) as such

```python
from bmtk.simulator.pointnet import cell_model

@cell_model(directive='nest', model_type='point_neuron')
def loadNESTModel(cell, template_name, dynamics_params):
    nest_nodes = ... # create and initialize cells
    return nest_nodes
```
To explain what is happening in this function
* The `@cell_model` decorator is placed right before our user defined function in order to tell bmtk that this is a special function that will be used in the creation of "nest" cells
  * The parameters tell BMTK that this function will be called for every group of cells with **model_template** of format `nest:...` and have **model_type** of point_neuron
* There are three parameters the BMTK will pass into this function
  * `cell` is copy of cell object and can be used to access all of an cell(s) properities (can be accessed like a dictionary)
  * `template` will be the name of the model (eg. izhikevich, glif_psc, hh_cond, etc)
  * `dynamics_params` will be a dictionary containing the contents of the json or hdf5 **dynamics_params**.
* Inside the function we should create our cell(s), usually using the NEST [Create](https://nest-simulator.readthedocs.io/en/v3.3/ref_material/pynest_apis.html#module-nest.lib.hl_api_nodes) function.
* You must return the results of the Create function, usually of list of ids, back to bmtk.

During the initialization process BMTK will call this custom function. It tries to do so in batches, often initializing cells with the same models and signitures in batches.


<div class="alert alert-block alert-info">
⚠️ WARNING: Overwriting loadNESTModel can be dangerous
</div>

If you're network has many different models then overwritting `loadNESTModel` to deal with only a subset of the cells can have unintended side-effects. However bmtk lets you create specific load methods for different models


#### Overriding Izhikeich model

For example, let's say we want to modify and adjust parameters on-the-fly (eg. we don't want to have to rebuild the network every time) but only for the `izhikevich` models. Specially we want to add jitter to the "d" parameter which won't exists for other model types. To do so we add a function `loadIzhikevich` and in the `@cell_model` parameters make sure that it only applies to `nest:Izhikevich` models, so that all other model types gets created in the default manner.

We add the following function

In [9]:
from bmtk.simulator.pointnet import cell_model
from bmtk.simulator.pointnet.io_tools import io
import nest

@cell_model(directive='nest:izhikevich', model_type='point_neuron')
def loadIzhikevich(cell, template_name, dynamics_params):
    nodes = nest.Create(template_name, cell.n_nodes, dynamics_params)
    
    d_orig = nest.GetStatus(nodes, 'd')
    jitter = cell['jitter']
    d_noisy = d_orig*jitter
    nest.SetStatus(nodes, {'d': d_noisy})
    io.log_info(f'Modifying the parameters of {cell.n_nodes} {template_name} neurons.')
    return nodes

* First we create the cells as normal using the `Create` method. Then we use the `nest.GetStatus` and `nest.SetStatus` to update the "d" parameters for each cell. It is sometimes possible to alter the way `Create()` method is called, but in general it is best to use [`GetStatus`](https://nest-simulator.readthedocs.io/en/v3.3/ref_material/pynest_apis.html#module-nest.lib.hl_api_info) to alter parameters
* The network has a jitter parameter which we can multiple by the default "d" value and is stored in the hdf5 file. We call `cell[jitter]` to get this value as an array, and can use `cell` to fetch other cell attributes.
* As an optional measure we add some logging. This is a good sanity check to make sure our funnction is being called as it should.

After we added our custom function we can load PointNet in the normal manner:

In [10]:
from bmtk.simulator import pointnet

configure = pointnet.Config.from_json('config.iz_updated.json')
configure.build_env()

network = pointnet.PointNetwork.from_config(configure)
sim = pointnet.PointSimulator.from_config(configure, network)
sim.run()

2024-10-21 15:11:51,907 [INFO] Created log file


INFO:Created log file


2024-10-21 15:11:51,930 [INFO] Batch processing nodes for net/0.


INFO:Batch processing nodes for net/0.


2024-10-21 15:11:51,938 [INFO] Batch processing nodes for virt_exc/0.


...

INFO:Starting Simulation


2024-10-21 15:11:52,297 [INFO] Simulation finished, finalizing results.


INFO:Simulation finished, finalizing results.


2024-10-21 15:11:52,316 [INFO] Done.


INFO:Done.


## 4. Example: Switching between synaptic (and electrical) models <a class="anchor" id="example-switching-models">

Similar to cell models, PointNet also allows modelers to switch different types of synapses between simulations. Like cells, SONATA groups synaptic and electrical connections (eg edges) by type and parameters, and you can have a network where different connections have different models. 

And like with the cells, the type of edge models in a network are controlled by the SONATA **model_template** and **dynamics_params** attributes. In the previous examples we have been using the default `static_synapse`, but the NEST simulator has a variety of different [built-in synaptic model](https://nest-simulator.readthedocs.io/en/v3.3/models/index.html#synapse) that can be readily taken advantage of in BMTK. If we wanted to replace it with one of the sypases that support [short-term plasticity](https://nest-simulator.readthedocs.io/en/v3.3/models/tsodyks2_synapse.html) we would modify the build network `add_edges` call to:
```python

net.add_edges(
    ...
    model_template='tsodysk2_synapse',
    dynamics_params='tsodysk2.syn_exc.json'
    ...
)
```
* unlike with cell, the SONATA format does not require a `nest:` schema prefix for the **model_template**
* *tsodysk2.syn_exc.json* is a .json file that is saved in the "synaptic_models_dir", and each synapse model will have different parameter that will need adjusted by the modeler.

Or if the modeler is unable to rebuild the network from scratch, another option for switching between model is to open the *network/\*edge_types.csv* file in a text editor and switch the **model_template** and **dynamics_params** values for the appropiate rows:

In [11]:
import pandas as pd

pd.read_csv('network_stp_syns/net_node_types.csv', sep=' ')

,node_type_id,model_template,model_type,dynamics_params
0,100,nest:izhikevich,point_neuron,custom_model_params.izhikevich.json
